In [2]:
 import keras

In [3]:
 !pip install syft==0.2.9 tensorflow_privacy==0.7.3

     |████████████████████████████████| 433 kB 4.7 MB/s 
     |████████████████████████████████| 251 kB 52.9 MB/s 
     |████████████████████████████████| 79 kB 6.5 MB/s 
     |████████████████████████████████| 484 kB 35.4 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 449 kB 37.9 MB/s 
     |████████████████████████████████| 66 kB 3.9 MB/s 
     |████████████████████████████████| 9.0 MB 16.2 MB/s 
     |████████████████████████████████| 126 kB 48.2 MB/s 
     |████████████████████████████████| 1.8 MB 35.8 MB/s 
     |████████████████████████████████| 753.4 MB 7.9 kB/s 
     |████████████████████████████████| 2.0 MB 33.4 MB/s 
     |████████████████████████████████| 57 kB 5.4 MB/s 
     |████████████████████████████████| 20.1 MB 52.8 MB/s 
     |████████████████████████████████| 73 kB 1.9 MB/s 
     |████████████████████████████████| 200 kB 45.8 MB/s 
     |████████████████████████████████| 4.0 MB 48.6 MB/s 
     |█████████████████

Defines Loss Function

In [1]:
from absl import logging
import collections
import tensorflow as tf

# from tensorflow_privacy.privacy.analysis import privacy_ledger
from tensorflow_privacy.privacy.dp_query import gaussian_query

def make_optimizer_class(cls):
  """Constructs a DP optimizer class from an existing one."""
  parent_code = tf.compat.v1.train.Optimizer.compute_gradients.__code__
  child_code = cls.compute_gradients.__code__
  GATE_OP = tf.compat.v1.train.Optimizer.GATE_OP  # pylint: disable=invalid-name
  if child_code is not parent_code:
    logging.warning(
        'WARNING: Calling make_optimizer_class() on class %s that overrides '
        'method compute_gradients(). Check to ensure that '
        'make_optimizer_class() does not interfere with overridden version.',
        cls.__name__)

  class DPOptimizerClass(cls):
    """Differentially private subclass of given class cls."""

    _GlobalState = collections.namedtuple(
      '_GlobalState', ['l2_norm_clip', 'stddev'])
    
    def __init__(
        self,
        dp_sum_query,
        num_microbatches=None,
        unroll_microbatches=False,
        *args,  # pylint: disable=keyword-arg-before-vararg, g-doc-args
        **kwargs):
      """Initialize the DPOptimizerClass.

      Args:
        dp_sum_query: DPQuery object, specifying differential privacy
          mechanism to use.
        num_microbatches: How many microbatches into which the minibatch is
          split. If None, will default to the size of the minibatch, and
          per-example gradients will be computed.
        unroll_microbatches: If true, processes microbatches within a Python
          loop instead of a tf.while_loop. Can be used if using a tf.while_loop
          raises an exception.
      """
      super(DPOptimizerClass, self).__init__(*args, **kwargs)
      self._dp_sum_query = dp_sum_query
      self._num_microbatches = num_microbatches
      self._global_state = self._dp_sum_query.initial_global_state()
      # TODO(b/122613513): Set unroll_microbatches=True to avoid this bug.
      # Beware: When num_microbatches is large (>100), enabling this parameter
      # may cause an OOM error.
      self._unroll_microbatches = unroll_microbatches

    def compute_gradients(self,
                          loss,
                          var_list,
                          gate_gradients=GATE_OP,
                          aggregation_method=None,
                          colocate_gradients_with_ops=False,
                          grad_loss=None,
                          gradient_tape=None,
                          curr_noise_mult=0,
                          curr_norm_clip=1):

      self._dp_sum_query = gaussian_query.GaussianSumQuery(curr_norm_clip, 
                                                           curr_norm_clip*curr_noise_mult)
      self._global_state = self._dp_sum_query.make_global_state(curr_norm_clip, 
                                                                curr_norm_clip*curr_noise_mult)
      

      # TF is running in Eager mode, check we received a vanilla tape.
      if not gradient_tape:
        raise ValueError('When in Eager mode, a tape needs to be passed.')

      vector_loss = loss()
      if self._num_microbatches is None:
        self._num_microbatches = tf.shape(input=vector_loss)[0]
      sample_state = self._dp_sum_query.initial_sample_state(var_list)
      microbatches_losses = tf.reshape(vector_loss, [self._num_microbatches, -1])
      sample_params = (self._dp_sum_query.derive_sample_params(self._global_state))

      def process_microbatch(i, sample_state):
        """Process one microbatch (record) with privacy helper."""
        microbatch_loss = tf.reduce_mean(input_tensor=tf.gather(microbatches_losses, [i]))
        grads = gradient_tape.gradient(microbatch_loss, var_list)
        sample_state = self._dp_sum_query.accumulate_record(sample_params, sample_state, grads)
        return sample_state
    
      for idx in range(self._num_microbatches):
        sample_state = process_microbatch(idx, sample_state)

      if curr_noise_mult > 0:
        grad_sums, self._global_state = (self._dp_sum_query.get_noised_result(sample_state, self._global_state))
      else:
        grad_sums = sample_state

      def normalize(v):
        return v / tf.cast(self._num_microbatches, tf.float32)

      final_grads = tf.nest.map_structure(normalize, grad_sums)
      grads_and_vars = final_grads#list(zip(final_grads, var_list))
    
      return grads_and_vars

  return DPOptimizerClass


def make_gaussian_optimizer_class(cls):
  """Constructs a DP optimizer with Gaussian averaging of updates."""

  class DPGaussianOptimizerClass(make_optimizer_class(cls)):
    """DP subclass of given class cls using Gaussian averaging."""

    def __init__(
        self,
        l2_norm_clip,
        noise_multiplier,
        num_microbatches=None,
        ledger=None,
        unroll_microbatches=False,
        *args,  # pylint: disable=keyword-arg-before-vararg
        **kwargs):
      dp_sum_query = gaussian_query.GaussianSumQuery(
          l2_norm_clip, l2_norm_clip * noise_multiplier)

      if ledger:
        dp_sum_query = privacy_ledger.QueryWithLedger(dp_sum_query,
                                                      ledger=ledger)

      super(DPGaussianOptimizerClass, self).__init__(
          dp_sum_query,
          num_microbatches,
          unroll_microbatches,
          *args,
          **kwargs)

    @property
    def ledger(self):
      return self._dp_sum_query.ledger

  return DPGaussianOptimizerClass
GradientDescentOptimizer = tf.compat.v1.train.GradientDescentOptimizer
DPGradientDescentGaussianOptimizer_NEW = make_gaussian_optimizer_class(GradientDescentOptimizer)

In [2]:
#import keras
#import tensorflow as tf
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer
from keras.datasets import mnist,cifar100
import numpy as np

# (mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = cifar100.load_data()

mnist_x_train = mnist_x_train.astype( np.float32 ) / 255
mnist_x_test = mnist_x_test.astype( np.float32 ) / 255

mnist_x_train = mnist_x_train.reshape( -1, 32, 32, 3)
mnist_x_test = mnist_x_test.reshape( -1, 32, 32, 3)

mnist_y_train = tf.keras.utils.to_categorical(mnist_y_train)#keras.utils.to_categorical( mnist_y_train )
mnist_y_test = tf.keras.utils.to_categorical(mnist_y_test)#keras.utils.to_categorical( mnist_y_test )

EPOCHS = 10
BATCH_SIZE = 500

model = tf.keras.models.Sequential()
model.add( tf.keras.layers.Conv2D( 32, kernel_size=(3, 3), activation='relu', input_shape=mnist_x_train.shape[ 1: ]  ) )
model.add( tf.keras.layers.MaxPooling2D( pool_size=(2, 2) ) )
model.add( tf.keras.layers.Conv2D( 64, kernel_size=(3, 3), activation='relu' ) )
model.add( tf.keras.layers.Flatten() )
model.add( tf.keras.layers.Dense( 128, activation='relu' ) )
model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )
optimizer = DPGradientDescentGaussianOptimizer_NEW(
   learning_rate = 0.0001,
   l2_norm_clip = 1.7,
   noise_multiplier = 1.0,
   num_microbatches = 150)

loss = tf.keras.losses.CategoricalCrossentropy( from_logits=True, reduction=tf.losses.Reduction.NONE )

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.fit(mnist_x_train, mnist_y_train,epochs= EPOCHS, steps_per_epoch=50000/500)

print( 'test acc:', model.evaluate( mnist_x_test, mnist_y_test, batch_size=150 ) )

eps = compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=60000, batch_size=150, noise_multiplier=1.3, epochs=5, delta=4e-5)
print( 'epsilon: ', eps )


169017344/169001437 [==============================] - 2s 0us/step
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


100/100 [==============================] - 15s 28ms/step - loss: 4.5859 - accuracy: 0.0177
Epoch 2/10
100/100 [==============================] - 3s 27ms/step - loss: 4.4502 - accuracy: 0.0396
Epoch 3/10
100/100 [==============================] - 3s 27ms/step - loss: 4.1923 - accuracy: 0.0692
Epoch 4/10
100/100 [==============================] - 3s 27ms/step - loss: 3.9909 - accuracy: 0.0981
Epoch 5/10
100/100 [==============================] - 3s 27ms/step - loss: 3.8685 - accuracy: 0.1160
Epoch 6/10
100/100 [==============================] - 3s 27ms/step - loss: 3.7782 - accuracy: 0.1325
Epoch 7/10
100/100 [==============================] - 3s 27ms/step - loss: 3.6983 - accuracy: 0.1496
Epoch 8/10
100/100 [==============================] - 3s 27ms/step - loss: 3.6197 - accuracy: 0.1609
Epoch 9/10
100/100 [==============================] - 3s 27ms/step - loss: 3.5431 - accuracy: 0.1738
Epoch 10/10
67/67 [==============================] - 1s 8ms/step - loss: 3.5477 - accuracy: 0.1749
te

In [3]:
import keras
import tensorflow as tf
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer
from keras.datasets import mnist,cifar100
import numpy as np

# (mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = cifar100.load_data()

mnist_x_train = mnist_x_train.astype( np.float32 ) / 255
mnist_x_test = mnist_x_test.astype( np.float32 ) / 255

mnist_x_train = mnist_x_train.reshape( -1, 32, 32, 3)
mnist_x_test = mnist_x_test.reshape( -1, 32, 32, 3)

mnist_y_train = tf.keras.utils.to_categorical(mnist_y_train)#keras.utils.to_categorical( mnist_y_train )
mnist_y_test = tf.keras.utils.to_categorical(mnist_y_test)#keras.utils.to_categorical( mnist_y_test )

EPOCHS = 10
BATCH_SIZE = 250

model = tf.keras.models.Sequential()
model.add( tf.keras.layers.Conv2D( 32, kernel_size=(3, 3), activation='relu', input_shape=mnist_x_train.shape[ 1: ]  ) )
model.add( tf.keras.layers.MaxPooling2D( pool_size=(2, 2) ) )
model.add( tf.keras.layers.Conv2D( 64, kernel_size=(3, 3), activation='relu' ) )
model.add( tf.keras.layers.Flatten() )
model.add( tf.keras.layers.Dense( 128, activation='relu' ) )
model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )
optimizer=DPGradientDescentGaussianOptimizer_NEW(
   learning_rate = 0.1,
   l2_norm_clip = 1.2,
   noise_multiplier = 1.2,
   num_microbatches = 100)

loss = tf.keras.losses.CategoricalCrossentropy( from_logits=True, reduction=tf.losses.Reduction.NONE )

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.fit(mnist_x_train, mnist_y_train,epochs= EPOCHS, steps_per_epoch=50000/500)

print( 'test acc:', model.evaluate( mnist_x_test, mnist_y_test, batch_size=250 ) )

eps = compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=60000, batch_size=250, noise_multiplier=1.3, epochs=10, delta=4e-5)
print( 'epsilon: ', eps )


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


100/100 [==============================] - 3s 28ms/step - loss: 750972817113088.0000 - accuracy: 0.0098
Epoch 2/10
100/100 [==============================] - 3s 27ms/step - loss: 4.8901 - accuracy: 0.0096
Epoch 3/10
100/100 [==============================] - 3s 27ms/step - loss: 4.6396 - accuracy: 0.0094
Epoch 4/10
100/100 [==============================] - 3s 27ms/step - loss: 4.6390 - accuracy: 0.0102
Epoch 5/10
100/100 [==============================] - 3s 27ms/step - loss: 4.6399 - accuracy: 0.0102
Epoch 6/10
100/100 [==============================] - 3s 26ms/step - loss: 4.6389 - accuracy: 0.0098
Epoch 7/10
100/100 [==============================] - 3s 27ms/step - loss: 4.6388 - accuracy: 0.0095
Epoch 8/10
100/100 [==============================] - 3s 27ms/step - loss: 4.6401 - accuracy: 0.0094
Epoch 9/10
100/100 [==============================] - 3s 27ms/step - loss: 4.6403 - accuracy: 0.0096
Epoch 10/10
40/40 [==============================] - 1s 9ms/step - loss: 4.6342 - accura